In [ ]:
import pyodbc
import json
import base64
from cryptography.fernet import Fernet
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import logging
import os

# --- Configuration variables ---
SERVER = 'PFDusSQL,50432'
DATABASE = 'fet_test'
SQL_QUERY_PATH = r'\\V-Processes\\ERP_Queries\\SKU_Carton_4_QS.sql'
CSV_FILENAME = r'\\pfduskommi\Promodoro.Export\Warehouse\SKU_Cartons_Purchase_4_QS.CSV'

# Configure logging to write to a file
logging.basicConfig(
    filename='warehouse_log.txt',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Gibt den Zugang zur ERP DB nur aus ENV zurück
def load_credential():
    username = os.getenv('DB_USER')
    password = os.getenv('DB_PASSWORD')
    if username and password:
        logging.info('Credentials loaded!')
        return username, password
    else:
        logging.critical('DB_USER and/or DB_PASSWORD environment variables are not set!')
        raise EnvironmentError('DB_USER and/or DB_PASSWORD environment variables are not set!')

def load_query(file_path):
    try:
        with open(file_path, 'r') as file:
            query = file.read()
        return query
    except FileNotFoundError as file_error:
        logging.error(f"SQL-Datei: {file_error}")

try:
    # Einlesen der Zugangsdaten nur aus ENV
    username, password = load_credential()

    # Angabe des Servers und der Datenbank
    connection_string = f'mssql+pyodbc://{username}:{password}@{SERVER}/{DATABASE}?driver=SQL+Server'
    sql_query = load_query(SQL_QUERY_PATH)

    engine = create_engine(connection_string)
    with engine.connect() as conn:
        df = pd.read_sql_query(sql_query, conn)
        df['Gewicht_kg'] = df['Gewicht_kg'].astype(str)
        df.to_csv(CSV_FILENAME, sep=';', index=False, encoding='windows-1252')
    logging.info('CSV export successful.')

except FileNotFoundError as fnf_error:
    logging.error(f"Datei nicht gefunden: {fnf_error}")
except pyodbc.Error as db_error:
    logging.error(f"fehlerhafte Verbindung: {db_error}")
except Exception as e:
    logging.critical(f" {e}", exc_info=True)